# 🚀 FreqAI GPU Backtest (Google Colab)

**⚠️ IMPORTANT:** Enable GPU before running!

**Runtime → Change runtime type → Hardware accelerator → GPU (T4)**

---

## 📋 Configuration

In [ ]:
# ⚙️ Backtest Configuration - EDIT THESE VALUES
TIMERANGE = '20250901-20251012'  # Format: YYYYMMDD-YYYYMMDD
PAIRS = ['BTC/USDT:USDT']  # Add more pairs: ['BTC/USDT:USDT', 'ETH/USDT:USDT', 'SOL/USDT:USDT']

print(f'⏰ Timerange: {TIMERANGE}')
print(f'💱 Pairs: {PAIRS}')

## ✅ GPU Verification

In [ ]:
# Check GPU availability
import subprocess
import sys

print('🔍 Checking for GPU...')
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print('✅ GPU AVAILABLE!')
    print(result.stdout)
except Exception as e:
    print('❌ GPU NOT FOUND!')
    print('\n⚠️  Please enable GPU:')
    print('   Runtime → Change runtime type → Hardware accelerator → GPU (T4)')
    print('\n💡 After enabling GPU, use: Runtime → Restart runtime, then run all cells again')
    sys.exit(1)

## 📦 Install Dependencies

In [ ]:
%%capture
# Install TA-Lib binary (faster than building from source)
!pip install ta-lib-binary

# Install Freqtrade with FreqAI support
!pip install freqtrade[freqai]

## 📥 Clone Repository

In [ ]:
# Clone the repository
!rm -rf freqai-futures-strategy
!git clone https://github.com/aminak58/freqai-futures-strategy.git
%cd freqai-futures-strategy

print('\n✅ Repository cloned successfully!')

## 📊 Download Historical Data

In [ ]:
# Download data for specified pairs
pairs_arg = ' '.join(PAIRS)

!freqtrade download-data \
    --exchange binance \
    --pairs {pairs_arg} \
    --timeframes 5m 15m 1h \
    --days 365 \
    --trading-mode futures \
    --config config/config.json

print('\n✅ Data download completed!')

## 🚀 Run GPU-Accelerated Backtest

In [ ]:
# Run backtest with LightGBM on GPU
!freqtrade backtesting \
    --strategy FreqAIHybridStrategy \
    --config config/config.json \
    --freqaimodel LightGBMRegressorMultiTarget \
    --timerange {TIMERANGE} \
    --export trades

print('\n✅ Backtest completed!')

## 📈 Display Results

In [ ]:
import json
from pathlib import Path

results_file = Path('user_data/backtest_results/backtest-result.json')

if results_file.exists():
    with open(results_file) as f:
        data = json.load(f)
        
        if 'strategy' in data and 'FreqAIHybridStrategy' in data['strategy']:
            stats = data['strategy']['FreqAIHybridStrategy']
            
            print('\n' + '='*70)
            print('📈 BACKTEST RESULTS - FreqAI Hybrid Strategy')
            print('='*70)
            print(f"\n💰 Total Profit: {stats.get('profit_total_abs', 'N/A')} USDT ({stats.get('profit_total', 'N/A')}%)")
            print(f"📊 Sharpe Ratio: {stats.get('sharpe', 'N/A')}")
            print(f"📉 Max Drawdown: {stats.get('max_drawdown', 'N/A')}%")
            print(f"📉 Max Drawdown Abs: {stats.get('max_drawdown_abs', 'N/A')} USDT")
            print(f"\n🎯 Total Trades: {stats.get('total_trades', 'N/A')}")
            print(f"✅ Wins: {stats.get('wins', 'N/A')}")
            print(f"❌ Losses: {stats.get('losses', 'N/A')}")
            print(f"📊 Win Rate: {stats.get('winrate', 'N/A')}%")
            print(f"\n⏱️  Avg Trade Duration: {stats.get('avg_duration', 'N/A')}")
            print(f"💵 Avg Profit: {stats.get('avg_profit', 'N/A')}%")
            print(f"🔝 Best Trade: {stats.get('best_pair', {}).get('profit_ratio', 'N/A')}%")
            print(f"📉 Worst Trade: {stats.get('worst_pair', {}).get('profit_ratio', 'N/A')}%")
            print('\n' + '='*70)
            
            # Show per-pair results if available
            if 'results_per_pair' in stats:
                print('\n📊 Results per Pair:')
                print('-' * 70)
                for pair_stats in stats['results_per_pair']:
                    if pair_stats['key'] != 'TOTAL':
                        print(f"{pair_stats['key']:20} | Trades: {pair_stats['trades']:3} | "
                              f"Profit: {pair_stats['profit_total_abs']:8.2f} USDT ({pair_stats['profit_total']:6.2f}%)")
                print('='*70)
        else:
            print('❌ Strategy results not found in backtest output!')
            print('Available keys:', list(data.keys()))
else:
    print('❌ Results file not found!')
    print(f'Expected location: {results_file.absolute()}')

## 💾 Download Results (Optional)

In [ ]:
# Download backtest results to your computer
from google.colab import files

results_file = Path('user_data/backtest_results/backtest-result.json')
if results_file.exists():
    files.download(str(results_file))
    print('✅ Results downloaded!')
else:
    print('❌ No results file to download')